# Clean Notebook for Webscraping of Blueground

Clean means: You can start from top an run to bottom without an error

# We load the libraries

In [18]:
# import all libraries that are needed

import requests                 # requests on websites
from bs4 import BeautifulSoup   # html parsing

import pandas as pd             # pandas for data frame
import numpy as np              # numpy for array

import math                     # for math methods
import time                     # for sleep timer

from sql_functions import *     # functions from file for upload on schema
import psycopg2                 # for upload on engine

import datetime as dt           # for the csv file with the current date and time

# libraries that are not needed right now
import re

# we create empty dataframes

In [19]:
# create an empty dataframe
df_full = pd.DataFrame()
df_object = pd.DataFrame()
df_search = pd.DataFrame()

# we set up the link and stuff for the loop

Test: Links: 

https://www.theblueground.com/furnished-apartments-london-uk?currency=GBP&language=en&offset=10&items=18

https://www.theblueground.com/furnished-apartments-london-uk?subArea=bayswater,bermondsey,bromley-by-bow,camden-town,canary-wharf,charing-cross,chelsea,city-of-london,clerkenwell,covent-garden,croydon,earls-court,farringdon,fitzrovia,fulham,hackney,hammersmith,highgate,holborn,islington,kensington,kentish-town,kings-cross,knightsbridge,ladbroke-grove,limehouse,maida-vale,marylebone,mayfair,notting-hill,old-street,paddington,piccadilly,pimlico,queens-park,shoreditch,soho,south-bank,south-kensington,southwark,stockwell,vauxhall,walthamstow,wandsworth,wapping,waterloo,westminster,whitechapel-brick-lane&currency=GBP&language=en&

offset=
2

&items=18"

In [20]:
weblink = 'https://www.theblueground.com/furnished-apartments-london-uk?subArea=camden-town,city-of-london,westminster,hackney,hammersmith,fulham,islington,kensington,chelsea,stockwell,vauxhall,waterloo,wapping,whitechapel-brick-lane,limehouse,bromley-by-bow&currency=GBP&language=en&'

In [21]:
# set up the link to the website

#weblink = 'https://www.theblueground.com/furnished-apartments-london-uk?currency=GBP&language=en&'

# set up the first page to scrape
pagesite = 0 # the website will start with 0 / # we set it to 10 to test the code

# create an empty list to store the blank slates
blank_slates_lst = [] 

# set the stop condition
stop_loop = "We’re sorry! We can’t seem to find any apartments that match your search." 

# check the link
print((weblink +  f'offset={ pagesite }&items=18'))


https://www.theblueground.com/furnished-apartments-london-uk?subArea=camden-town,city-of-london,westminster,hackney,hammersmith,fulham,islington,kensington,chelsea,stockwell,vauxhall,waterloo,wapping,whitechapel-brick-lane,limehouse,bromley-by-bow&currency=GBP&language=en&offset=0&items=18


## we need the following structure for the dataframe:
+ platform_id	
+ platform	
+ neighbourhood	
+ property_type	
+ bedrooms	
+ bathroom	
+ price_pcm	
+ title	
+ furnished	
+ available_from	
+ size

# Neighbourhoods Ukio wants from us

+ Camden
+ City of London
+ City of Westminster
+ Hackney
+ Hammersmith and Fulham
+ Islington
+ Kensington and Chelsea
+ Lambeth --> Vauxhall
+ Tower Hamlets

### List of Neighbourhoods from Blueground we have to use


+ camden-town
+ city-of-london
+ westminster
+ hackney
+ hammersmith
+ fulham
+ islington
+ kensington
+ chelsea
+ stockwell
+ vauxhall
+ waterloo
+ bromley-by-bow
+ limehouse
+ wapping
+ whitechapel-brick-lane

This gives us this a selection of 83 flats (27.09.2022)

https://www.theblueground.com/api/furnished-apartments-london-uk?subArea=camden-town,city-of-london,westminster,hackney,hammersmith,fulham,islington,kensington,chelsea,stockwell,vauxhall,waterloo,wapping,whitechapel-brick-lane,limehouse,bromley-by-bow&currency=GBP&language=en&
offset= 1 &items=18

-----

# we load all the functions

platform_id --> from Detail-Page --> currently not working
we get the ID from the last part of the URL

In [22]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html

def get_blueground_id(df_search):
    # create an empty list to store the blueground_id
    blueground_id_lst = []
    # get the blueground_id from the dataframe and the column get_url_to_detail_page
    for blueground_id in df_search.get_url_to_detail_page: # we get the url from the dataframe
        blueground_id_lst.append(blueground_id.rsplit('-', 1)[-1]) # we get the last part of the url
    return blueground_id_lst

platform --> we can add the Origin-Platform during the Loop, in this Notebook we have only "blueground"

neighbourhood

In [23]:
def get_neighborhoods(bs):
    neighborhood_names = bs.find_all("div", {"class":"name-place"})
    neighborhood_names_lst = (neighborhood_name.get_text() for neighborhood_name in neighborhood_names)
    neighborhood_names_lst = [neighborhood_name.strip() for neighborhood_name in neighborhood_names_lst]
    neighborhood_names_lst = [i.rsplit(',', 1)[-1] for i in neighborhood_names_lst]
    return neighborhood_names_lst

property_type 
on Blueground their is only "Studio" or "Apartment"

> later we have to change the Type  "Jr. Bedroom" to "Studio"

In [24]:
def get_property_type(bs):
    # get the property_type out of the apartment amenities
    lst_name = []
    property_typs = bs.find_all(class_="main-amenities")
    property_typs_lst = [property_type.get_text() for property_type in property_typs]
    property_typs_lst = [property_type.strip() for property_type in property_typs_lst]
    property_typs_lst = [i.split('o', 1)[0] for i in property_typs_lst]
    property_typs_lst = [i.replace('Bedr', 'Bedroom') for i in property_typs_lst]
    property_typs_lst = [i.replace('Studi', 'Studio') for i in property_typs_lst]
    property_typs_lst = [i.rsplit(' ', 1)[-1] for i in property_typs_lst]
    #property_typs_lst = [i.replace('Bedroom', 'Apartment') for i in property_typs_lst]
    return property_typs_lst

bedrooms

In [25]:
def get_bedrooms(bs):
    # get the main-amenities of the apartments
    lst_name = []
    bedrooms = bs.find_all(class_="main-amenities")
    bedrooms_lst = [bedroom.get_text() for bedroom in bedrooms]
    bedrooms_lst = [bedroom.strip() for bedroom in bedrooms_lst]
    bedrooms_lst = [i.split('o', 1)[0] for i in bedrooms_lst]
    bedrooms_lst = [i.replace('Bedr', 'Bedroom') for i in bedrooms_lst]
    bedrooms_lst = [i.replace('Studi', 'Studio') for i in bedrooms_lst]
    return bedrooms_lst

bathroom

In [26]:
def get_bathroom(bs):
    # get the main-amenities of the apartments
    lst_name = []
    bathrooms= bs.find_all(class_="main-amenities")
    bathrooms_lst = [bathroom.get_text() for bathroom in bathrooms]
    bathrooms_lst = [bathroom.strip() for bathroom in bathrooms_lst]
    bathrooms_lst = [i.rsplit('o', 1)[-1] for i in bathrooms_lst]
    bathrooms_lst = [i.replace('m', '') for i in bathrooms_lst]

    return bathrooms_lst

price_pcm (price per month)

In [27]:
def get_price_pcm(bs):
    # get the price per month
    lst_name = []
    prices = bs.find_all(class_= "price__amount")
    for price in prices:
        lst_name.append(
            price.get_text()
                .strip()
                .replace(',', '')
        )
    return lst_name

 title

In [28]:
def get_object_title(bs):
    # get the names of all the apartments
    lst_name = []
    object_titles = bs.find_all(class_="listing-name")
    for object_title in object_titles:
        lst_name.append(
            object_title.get_text()
                .strip()
        )
    return lst_name


furnished --> on Blueground we have only furnished Studios/Apartments

available_from

In [29]:
def get_availability(bs):
    # get the availability of the apartments
    lst_availability = []
    availability = bs.find_all(class_="availability__date")
    for avail in availability:
        lst_availability.append(
            avail.get_text()
                .strip()
        )
    return lst_availability


size

-----

# We need this for extra work

url

In [30]:
def get_url_to_detail_page(bs, maximus):
    url_lst = []
    count = 0
    while count <= int(maximus):
        
        # this will get us the link to the detail page
        class_with_link = bs.find_all(class_="ui-image-carousel")
        # with the [] we can select the elment we want to get
        for a in class_with_link[count].find_all('a', href=True):
            url_lst.append(a['href'])
        count += 1
    #print(url_lst)
    return url_lst

----

# the Loop to grab everything

In [31]:
# https://flexiple.com/python/check-if-list-is-empty-python/
# Solution 3: Using len() function
# The len() function returns the number of items in a list. If the list is empty, it returns 0.
while len(blank_slates_lst) == 0: # start and endpoint of the for-loop
    # pause the loop for 3 seconds to reduce the load on the server
    time.sleep(3)

    # ----------------- get the data from the website -----------------
    # get the content of the website
    page = requests.get(weblink +  f'offset={ pagesite }&items=18')
    # parse the html and save it into a BeautifulSoup instance
    html = page.content
    bs = BeautifulSoup(html, 'html.parser')

    # ----------------- insert the data into the dataframe -----------------
    # create a pandas dataframe for the names and prices
    blueground_dict = {
        #'platform_id', --> we get this from the detail page
        'platform': 'blueground',
        'neighbourhood': get_neighborhoods(bs),
        'property_type': get_property_type(bs),
        'bedrooms': get_bedrooms(bs),
        'bathroom': get_bathroom(bs),
        'price_pcm': get_price_pcm(bs),
        'title': get_object_title(bs),
        'furnished': 'furnished',	
        'available_from': get_availability(bs),
        #'size': , --> we get this from the detail page
        'title': get_object_title(bs)
        }
    
    # ----------------- work with the datafram to get the URL and the ID -----------------
    # we now have a dataframe, we can use this to get a counter for the URL
    df_page = pd.DataFrame(blueground_dict)

    # we create a variable to store the number of rows in the dataframe
    maximus = df_page.index.max() # we give it the max value of the index

    # we create a list to store the urls
    # we check if we reached the end of the existing df, to stop and not get an error
    if np.isnan(maximus):
        break
    else:
        df_page['get_url_to_detail_page'] = pd.Series(get_url_to_detail_page(bs, maximus))
    # we can now add the dataframe to the full dataframe
    df_search = df_search.append(df_page, ignore_index=True)


    # ----------------- check if we reached the end of the pages -----------------
    blank_slates = bs.find_all(class_="blank-slate__criteria")
    blank_slates_lst = (blank_slate.get_text() for blank_slate in blank_slates)
    blank_slates_lst = [blank_slate.strip() for blank_slate in blank_slates_lst]

    # increase the pagesite by 1
    pagesite += 1


# ----------------- get the ID from URL -----------------

df_search['blueground_id'] = get_blueground_id(df_search)




#------- set the number of rows and column_width  to maximum ------- 
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_colwidth', None)

# ----------------- display the first 5 rows -----------------
display(df_search.head(5))
display(df_search.tail(5))

/var/folders/yg/z64xf8615mxbsw25_zldzjnm0000gn/T/ipykernel_16407/2627209534.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_search = df_search.append(df_page, ignore_index=True)
/var/folders/yg/z64xf8615mxbsw25_zldzjnm0000gn/T/ipykernel_16407/2627209534.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_search = df_search.append(df_page, ignore_index=True)
/var/folders/yg/z64xf8615mxbsw25_zldzjnm0000gn/T/ipykernel_16407/2627209534.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_search = df_search.append(df_page, ignore_index=True)
/var/folders/yg/z64xf8615mxbsw25_zldzjnm0000gn/T/ipykernel_16407/2627209534.py:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a fu

,platform,neighbourhood,property_type,bedrooms,bathroom,price_pcm,title,furnished,available_from,get_url_to_detail_page,blueground_id
0,blueground,Vauxhall,Bedroom,1 Bedroom,1 Bath,4350,"Legacy Building, Viaduct Gdns,",furnished,28 Sep 2022,/furnished-apartments-london-uk/london-vauxhall-191,191
1,blueground,Vauxhall,Bedroom,2 Bedroom,2 Bath,5520,"Embassy Gardens, Viaduct Gdns,",furnished,28 Sep 2022,/furnished-apartments-london-uk/london-vauxhall-174,174
2,blueground,Vauxhall,Bedroom,2 Bedroom,2 Bath,4480,"Sky Gardens, Wandsworth Rd,",furnished,30 Sep 2022,/furnished-apartments-london-uk/london-vauxhall-257,257
3,blueground,Vauxhall,Bedroom,1 Bedroom,1 Bath,4060,"Wandsworth Rd,",furnished,02 Oct 2022,/furnished-apartments-london-uk/london-vauxhall-159,159
4,blueground,Whitechapel/Brick Lane,Bedroom,1 Bedroom,1 Bath,3120,"Gunthorpe St,",furnished,05 Oct 2022,/furnished-apartments-london-uk/london-whitechapel-brick-lane-079,079


,platform,neighbourhood,property_type,bedrooms,bathroom,price_pcm,title,furnished,available_from,get_url_to_detail_page,blueground_id
69,blueground,Kensington,Bedroom,1 Bedroom,1 Bath,3770,"Old Brompton Rd,",furnished,02 Sep 2023,/furnished-apartments-london-uk/london-kensington-073,073
70,blueground,Waterloo,Studio,Studio,1 Bath,3210,"Walpole House, Westminster Bridge Rd,",furnished,17 Sep 2023,/furnished-apartments-london-uk/london-waterloo-262,262
71,blueground,Waterloo,Bedroom,2 Bedroom,2 Bath,5560,"West Block, Forum Magnum Square,",furnished,19 Sep 2023,/furnished-apartments-london-uk/london-waterloo-196,196
72,blueground,Whitechapel/Brick Lane,Bedroom,1 Bedroom,1 Bath,3550,"Cavell St,",furnished,27 Sep 2023,/furnished-apartments-london-uk/london-whitechapel-brick-lane-145,145
73,blueground,Hammersmith,Bedroom,1 Bedroom,1 Bath,4130,"Matcham House, Glenthorne Rd,",furnished,29 Nov 2023,/furnished-apartments-london-uk/london-hammersmith-205,205


--------

# Save the Data

In [32]:
# # create a .csv file with the current date and time
today = dt.datetime.today().strftime('%Y-%m-%d-%H-%M')
df_search.to_csv('data/bluebround_{}.csv'.format(today), sep='\t')
print("we have saved the new dataframe in a .csv file")

we have saved the new dataframe in a .csv file


----

# 1. Basic information about data - EDA

https://www.digitalocean.com/community/tutorials/exploratory-data-analysis-python

In [33]:
#Load the required libraries
import pandas as pd
import numpy as np
import seaborn as sns

In [34]:
#Basic information

df_search.info()

#Describe the data

df_search.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74 entries, 0 to 73
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   platform                74 non-null     object
 1   neighbourhood           74 non-null     object
 2   property_type           74 non-null     object
 3   bedrooms                74 non-null     object
 4   bathroom                74 non-null     object
 5   price_pcm               74 non-null     object
 6   title                   74 non-null     object
 7   furnished               74 non-null     object
 8   available_from          74 non-null     object
 9   get_url_to_detail_page  74 non-null     object
 10  blueground_id           74 non-null     object
dtypes: object(11)
memory usage: 6.5+ KB


,platform,neighbourhood,property_type,bedrooms,bathroom,price_pcm,title,furnished,available_from,get_url_to_detail_page,blueground_id
count,74,74,74,74,74,74,74,74,74,74,74
unique,1,16,2,5,4,58,51,1,58,74,74
top,blueground,Vauxhall,Bedroom,1 Bedroom,1 Bath,4060,"Lower Marsh,",furnished,15 Oct 2022,/furnished-apartments-london-uk/london-vauxhall-191,191
freq,74,16,71,38,47,4,8,74,5,1,1


# 2. Duplicate values

In [35]:
#Find the duplicates

df_search.duplicated().sum()

0

# 5. Find the Null values

In [36]:
#Find null values

df_search.isnull().sum()

platform                  0
neighbourhood             0
property_type             0
bedrooms                  0
bathroom                  0
price_pcm                 0
title                     0
furnished                 0
available_from            0
get_url_to_detail_page    0
blueground_id             0
dtype: int64

# 7. Know the datatypes

In [37]:
#Datatypes

df_search.dtypes

platform                  object
neighbourhood             object
property_type             object
bedrooms                  object
bathroom                  object
price_pcm                 object
title                     object
furnished                 object
available_from            object
get_url_to_detail_page    object
blueground_id             object
dtype: object

------

# Import the data frame to DBeaver

In [38]:
# import the data frame to DBeaver

# call the schema created for this project
schema = 'capstone_jmrs'
# get the function to connect to the database
engine = get_engine()

# give the table a unique name
table_name = 'blueground_df_complete'

# import the table to sql
if engine!=None:
    try:
        df_search.to_sql(name=table_name,
                        con=engine,
                        if_exists='replace',
                        schema=schema, 
                        index=False,
                        chunksize=5000, 
                        method='multi')
        print(f"The {table_name} table was imported successfully.")
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

The blueground_df_complete table was imported successfully.
